In [14]:
import numpy as np
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Input, Dropout, SimpleRNN
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import vgg16
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [15]:
#what data are we creating
raw_file='just_notes.txt'
weight_dir='note_weights/'
last_file=None
last_epoch=0

In [16]:
#reading in data from file
songs = []

with open(raw_file) as f:
    songs = f.read()
label = sorted(list(set(songs)))

In [17]:
#getting number of classes (one-hot encoding so this is number of unique chars)
NUM_CLASSES = len(label)
BATCH_SIZE = 30
SEQ_LEN = 25

In [18]:
#formatting training/validation data and labels
train_data = list(songs[:round(len(songs)*0.8)])
validation_data = list(songs[round(len(songs)*0.8):])

train_labels = list(map(lambda x: label.index(x), train_data))
validation_labels = list(map(lambda x: label.index(x), validation_data))

X = []
y = []
for i in range(len(train_data) - SEQ_LEN):
    X.append(train_labels[i:i+SEQ_LEN])
    y.append(train_labels[i+SEQ_LEN])
    
val_X = []
val_y = []
for i in range(len(validation_data) - SEQ_LEN):
    val_X.append(validation_labels[i:i+SEQ_LEN])
    val_y.append(validation_labels[i+SEQ_LEN])

X = np.reshape(X, (len(X),SEQ_LEN,1))
X = X / len(label)
y = np_utils.to_categorical(y)

val_X = np.reshape(val_X, (len(val_X), SEQ_LEN, 1))
val_y = np_utils.to_categorical(val_y)

In [ ]:
#creating model
model = Sequential()
layers = [1, 100, NUM_CLASSES]
model.add(SimpleRNN(input_shape=(SEQ_LEN,1), output_dim=layers[1], return_sequences=False))
model.add(Dense(layers[2], activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
if (last_file):
    model.load_weights(weight_dir+last_file)

In [ ]:
#training model
filepath=weight_dir + "weights.{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)

res = model.fit(X, y, nb_epoch=30, batch_size=BATCH_SIZE,
                validation_data=(X, y), callbacks=[checkpoint],
                initial_epoch=last_epoch)

Train on 197753 samples, validate on 197753 samples
Epoch 1/30
 28530/197753 [===>..........................] - ETA: 167s - loss: 3.0004 - acc: 0.2120